# Experiment IDs
There are multiple ways to uniquely identify an experiment:

- eID (uuid.UUID) : An experiment UUID object (or as a 32 hexadecimal hyphenated str)
- path (Path) : A pathlib ALF path of the form \<lab\>/Subjects/\<subject\>/\<date\>/\<number\>
- ref (str) : An experiment reference string of the form yyyy-mm-dd_n_subject
- url (str) : An remote http session path of the form \<lab\>/Subjects/\<subject\>/\<date\>/\<number\>
- np (int64) : An experiment UUID encoded as 2 int64s (deprecated)

Internally Alyx and ONE uses eID UUID objects to identify sessions.  One.search and OneAlyx.search_insertions return a
list of eID UUIDs. Session paths, URLs and ref strings are more readable, however the hyphenated 36 char str form of an
eID is also acceptable as input.

In [1]:
from uuid import UUID

from one.api import ONE
from one.alf.spec import is_session_path, is_uuid_string, is_uuid

one = ONE(base_url='https://openalyx.internationalbrainlab.org')

One.search returns experiment uuid strings

In [ ]:
eids = one.search(datasets='channels.brainLocationIds_ccf_2017.npy', date=['2022-08-01', '2022-09-01'])
assert is_uuid(eids[0])

eID strings can be easily converted to other forms

In [3]:
session_path = one.eid2path(eids[0])  # returns a pathlib.Path object
assert is_session_path(session_path)
print(f'Session {"exists" if session_path.exists() else "does not exist"} on disk')

uuid_str = str(eids[0])
assert is_uuid_string(uuid_str)

Session exists on disk


These conversion functions can except lists of experiment ids

In [4]:
ref_dict = one.eid2ref(eids)
assert len(ref_dict) == len(eids)
print(ref_dict[0])

{'subject': 'NR_0027', 'date': datetime.date(2022, 8, 23), 'sequence': 1}


ref strings can be sorted lexicographically (by date, number and subject in that order)

In [5]:
refs = sorted(one.dict2ref(ref_dict))
print(refs)

# Most ids can be interconverted also
eid = one.path2eid(
    one.ref2path(
        one.dict2ref(
            one.eid2ref(eids[0])
        )
    )
)
assert eid == eids[0]

['2022-08-02_1_UCLA049', '2022-08-12_1_UCLA049', '2022-08-16_1_UCLA048', '2022-08-19_1_NR_0027', '2022-08-20_1_NR_0027', '2022-08-22_1_NR_0027', '2022-08-23_1_NR_0027']


The `to_eid` method will convert any form to a UUID string or raise a ValueError if invalid:

In [6]:
ids = ['2019-12-10_1_KS023', 'ae8787b1-4229-4d56-b0c2-566b61a25b77', 'CSH_ZAD_029/2020-09-19/001']
print(one.to_eid(ids))

[UUID('aad23144-0e52-4eac-80c5-c4ee2decb198'), UUID('ae8787b1-4229-4d56-b0c2-566b61a25b77'), UUID('c7bd79c9-c47e-4ea5-aea3-74dda991b48e')]


One load functions can accept most kinds of experiment identifiers

In [9]:
filepath = one.load_dataset(eid, 'rightROIMotionEnergy.position.npy', download_only=True)
dset = one.load_dataset(session_path, 'rightROIMotionEnergy.position.npy')
dset = one.load_dataset(filepath, 'rightROIMotionEnergy.position.npy')

short_path = '/'.join(session_path.parts[-3:])  # 'subject/date/number'
dset = one.load_dataset(short_path, 'rightROIMotionEnergy.position.npy')

url = one.path2url(filepath)
dset = one.load_dataset(url, 'rightROIMotionEnergy.position.npy')
dset = one.load_dataset(ref_dict[0], 'rightROIMotionEnergy.position.npy')
dset = one.load_dataset(refs[2], 'rightROIMotionEnergy.position.npy')

(S3) F:\FlatIron\openalyx.internationalbrainlab.org\churchlandlab_ucla\Subjects\UCLA048\2022-08-16\001\alf\rightROIMotionEnergy.position.npy: 100%|██████████| 160/160 [00:00<00:00, 595B/s]


Likewise with other load methods...

In [12]:
obj = one.load_object(short_path, 'channels', attribute='brainLocationIds', collection='alf/probe00/pykilosort')

You can get information about an experiment from its ID using the `get_details` method:

In [13]:
info = one.get_details(eid)
info

{'subject': 'NR_0027',
 'lab': 'steinmetzlab',
 'projects': 'ibl_neuropixel_brainwide_01',
 'task_protocol': '_iblrig_tasks_ephysChoiceWorld6.6.2',
 'number': 1,
 'start_time': '2022-08-23T09:26:09',
 'url': 'https://openalyx.internationalbrainlab.org/sessions/ae8787b1-4229-4d56-b0c2-566b61a25b77',
 'local_path': WindowsALFPath('F:/FlatIron/openalyx.internationalbrainlab.org/steinmetzlab/Subjects/NR_0027/2022-08-23/001'),
 'date': datetime.date(2022, 8, 23)}

In online mode you get retrieve exhaustive session information:

In [14]:
assert not one.offline
info = one.get_details(eid, full=True)
print(info.keys())

dict_keys(['subject', 'users', 'location', 'procedures', 'lab', 'projects', 'type', 'task_protocol', 'number', 'start_time', 'end_time', 'narrative', 'parent_session', 'n_correct_trials', 'n_trials', 'url', 'extended_qc', 'qc', 'wateradmin_session_related', 'data_dataset_session_related', 'auto_datetime', 'id', 'json', 'probe_insertion', 'field_of_view', 'notes'])
